# $H_2O$ cooling rate profiles

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [45]:
import os
import sys
import shutil
import pprint
from pathlib import *

import pymongo
import json
import pandas as pd
import xarray as xr

from climatools.lblnew.bestfit_params import *
from climatools.atm import *
from climatools.parameters import *
from climatools.dataset import *
from climatools.spectrum import *

In [12]:
client = pymongo.MongoClient('localhost', 27017)

In [96]:
PATH = Path('./data')

## Parameters

In [13]:
MOLECULE = 'h2o'

In [14]:
atm = AtmComposition.cliradlw_nongreys()
BANDMOLS = [(mapband_new2old()[band], 'h2o') for band, absorbers in atm.gasinbands.items() if 'h2o' in absorbers]

In [15]:
ATMPROS = [('saw_trpwlayer', 257), ('trp_sawwlayer', 300)]

In [35]:
PARAMS = []
for band, absorber in BANDMOLS:
    for atmpro, tsfc in ATMPROS:
        param = kdist_params(molecule=absorber, band=band)
        param['commitnumber'] = '9dc25c5'
        param['atmpro'], param['tsfc'] = atmpro, tsfc
        param['option_compute_ktable'] = 0
        param['option_compute_btable'] = 0
        PARAMS.append(param)

## Cooling rate profiles from all spectral bands in Dataframe
These are also exported to excel files.

In [92]:
def cool_bands(*ps):
    for p in ps:
        data = p.modeldata_pymongo(collection=client.lblnew.bestfit_lw)
        cool = data.crd_cool.sum('g')
        yield cool
        
def cool_table(atmpro):
    ps = [p for p in PARAMS if p['atmpro'] == atmpro]
    ps = [LBLnewBestfitParam(**p) for p in ps]
    
    bands = [mapband_old2new()[p.band] for p in ps]
    ds = xr.concat(cool_bands(*ps), dim='band')
    ds.coords['band'] = bands
    
    df = ds['coolrg'].to_pandas().transpose()
    df['bands-total'] = df.sum(axis=1)
    df['layer'] = ds['layer'].values
    cs = list(df.columns)
    cs.remove('layer')
    df = df[['layer'] + cs]
    return df

In [123]:
list(PATH.iterdir())[0].resolve()

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param/sawtrp_swap_atmpros/data/cool_specialprofiles.xlsx')

In [110]:
os.makedirs('data', exist_ok=True)

In [115]:
with pd.ExcelWriter(PATH/'cool_specialprofiles.xlsx') as writer:
    for atmpro, tsfc in ATMPROS:
        cool_table(atmpro).to_excel(writer, sheet_name=atmpro)

# fin